In [1]:
import requests
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

In [6]:
codelist = ["ABAQ"]
benchmark = "ABQX"
start_date = "2016-03-13"
end_date = "2020-03-13"

In [8]:
# Load any files needed
with open('data/key.txt', 'r') as file:
    api_key = file.read()

In [23]:
# Pull data from quandl
indexdata = pd.DataFrame()

quandl_code = codelist
if benchmark != 'None':
    quandl_code = codelist + [benchmark]

for x in range(0,len(quandl_code)):
    quandl_request = (
    'https://www.quandl.com/api/v3/datasets/NASDAQOMX/'
    f'{quandl_code[x]}?start_date={start_date}&end_date={end_date}&api_key={api_key}'
    )
    
    response = requests.get(quandl_request).json()
    
    response_df = pd.DataFrame(response['dataset']['data'])
    response_df = response_df[[0,1]]
    if x == len(quandl_code)-1:
        response_df.columns = ['date','benchmark']
    else:
        response_df.columns = ['date',quandl_code[x]]
    
    if x == 0:
        indexdata = indexdata.append(response_df)
    else:
        indexdata = pd.merge(indexdata,response_df)

In [33]:
indexdata = indexdata.sort_values('date').reset_index(drop = True)
indexdata['ABAQreturn'] = (indexdata.ABAQ - indexdata.ABAQ.shift(1))/indexdata.ABAQ.shift(1)
indexdata['benchmarkreturn'] = (indexdata.benchmark - indexdata.benchmark.shift(1))/indexdata.benchmark.shift(1)

In [30]:
# For one set of returns, we can just compute the covariance between our benchmark and portfolio, 
# which should always be true as I will be compressing portfolio into one set of returns
covariance_matrix = indexdata[["ABAQreturn","benchmarkreturn"]].cov()
covariance_coefficient = covariance_matrix.iloc[0,1]
benchmark_variance = indexdata["benchmarkreturn"].var()
portfolio_beta = covariance_coefficient / benchmark_variance

0.9522401831028744

In [91]:
model = smf.ols(formula='ABAQreturn ~ benchmarkreturn', data=indexdata)
results = model.fit()

In [177]:
overview = pd.DataFrame(results.summary().tables[0])

In [178]:
temporary_overview = overview[[2,3]]
temporary_overview.columns = [0,1]

In [179]:
overview = pd.concat([overview[[0,1]],temporary_overview],ignore_index=True)
overview = overview.transpose()
overview = overview.drop([16,17], axis = 1)
overview.columns = overview.iloc[0]
overview = overview.drop([0], axis = 0)

In [113]:
parameters = pd.DataFrame(results.summary().tables[1])

In [186]:
parameters.columns = parameters.iloc[0]
parameters = parameters.drop(0,axis = 0)

In [115]:
metrics = pd.DataFrame(results.summary().tables[2])

In [196]:
temporary_metrics = metrics[[2,3]]
temporary_metrics.columns = [0,1]

In [197]:
metrics = pd.concat([metrics[[0,1]],temporary_metrics],ignore_index=True)
metrics = metrics.transpose()

In [199]:
metrics.columns = metrics.iloc[0]
metrics = metrics.drop([0], axis = 0)

In [240]:
frenchfama = pd.read_csv('data/ff5factordaily.CSV')

In [241]:
frenchfama = frenchfama[frenchfama['date']>start_date]
frenchfama = frenchfama[frenchfama['date']<=end_date].reset_index(drop=True)

In [250]:
regression_data = pd.concat([indexdata,frenchfama],axis = 1,join = 'inner')

In [261]:
model = smf.ols(formula='ABAQreturn ~ benchmarkreturn + SMB + HML + RMW + CMA', data=regression_data)
results = model.fit()

In [262]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             ABAQreturn   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 1.226e+05
Date:                Thu, 30 Apr 2020   Prob (F-statistic):               0.00
Time:                        20:57:36   Log-Likelihood:                 6156.1
No. Observations:                1005   AIC:                        -1.230e+04
Df Residuals:                     999   BIC:                        -1.227e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept       -7.091e-05   1.68e-05     -4.225      0.000      -0.000    -3.8e-05
benchmarkreturn     0.9511      0.001    685.489      0.000       0.948       0.954
SMB                 0.0001   3.53e-05      4.138      0.000    7.68e-05       0.000
HML                -0.0002   3.61e-05     -5.879      0.000      -0.000      -0.000
RMW              3.237e-05   4.97e-05      0.651      0.515   -6.52e-05       0.000
CMA                 0.0001   6.04e-05      2.319      0.021    2.16e-05       0.000
==============================================================================
Omnibus:                      609.424   Durbin-Watson:                   1.910
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            21588.483
Skew:                          -2.190   Prob(JB):                         0.00
Kurtosis:                      25.279   Cond. No.                         83.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""